## Getting and saving the cleaned twitter data
* TODO Describe data based on analysis done on cluster
* The code to get the data is in cleaned_twitter.py and other scripts. TODO Mention which other scripts.
* #TODO write comments and reasons for writing stuff

In [3]:
import pandas as pd
import os
from tqdm import tqdm
from pyspark import SparkConf, HiveContext
import pyspark.sql
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, lit, udf
import pyspark.sql.functions as functions
import pickle
import time

conf = SparkConf().setAppName("ADA-gcl")
sqlContext = SQLContext(sc)

## Load the parquet data
As the goal of our project evolved, we realized that we needed more columns than the ones we had already queried for the twitter dataset. Each time we query, we save parquet files by month. Here we combine all the different queries that we have done for different columns and clean the dataframe to get it iin a form we need for the visualisation.

In [7]:
path1 = 'file:///home/kirtan/Academics/EPFL/sem1/ADA/ADA-Project/twitter_data/cleaned/'
files1 = os.listdir('twitter_data/cleaned')

df1 = sqlContext.read.parquet(path1+files1[0])
df_twitter1 = df1

for f in tqdm(files1[1:]):
    temp = sqlContext.read.parquet(path1 + f)
    df_twitter1 = df_twitter1.unionAll(temp)  

In [6]:
# Number of tweets
df_twitter1.count()

10828070

In [7]:
df_twitter1.show()

+-------------------+--------+---------+--------------------+--------------------+--------------------+
|                 id|language|sentiment|            location|                main|                date|
+-------------------+--------+---------+--------------------+--------------------+--------------------+
|1467622550934000011|      en| POSITIVE|            Schenkon| @beknuesel Fast :-)|2016-07-04T08:51:23Z|
|1467661901184400101|      en| NEGATIVE|            Svizzera|@sebdivo @ildivoo...|2016-07-04T17:05:55Z|
|1467664145156400055|      en|  NEUTRAL|           Neuchâtel|Need you by my side.|2016-07-04T17:30:19Z|
|1467656305452100106|      en| POSITIVE|                null|One Nation. One L...|2016-07-04T16:16:34Z|
|1467672681723400019|      en| POSITIVE|         Orlando, FL|Happy Independenc...|2016-07-04T22:05:47Z|
|1467663376923900019|      en|  NEUTRAL|         Switzerland|@Moreterm Full of...|2016-07-04T17:21:37Z|
|1467670928189100298|      en|  NEUTRAL|         Switzerland|@An

In [8]:
df_twitter1 = df_twitter1.where(col("language").isin({"en", "de", "fr"}))
df_twitter1 = df_twitter1.filter(df.sentiment != 'null')
df_twitter1.count()

8719750

In [9]:
# See main languages
df_twitter1.groupBy(['language', 'sentiment']).count().sort(col("language").desc()).collect()

[Row(language='fr', sentiment='NEUTRAL', count=2018350),
 Row(language='fr', sentiment='POSITIVE', count=53314),
 Row(language='fr', sentiment='NEGATIVE', count=8254),
 Row(language='en', sentiment='POSITIVE', count=1423976),
 Row(language='en', sentiment='NEGATIVE', count=676615),
 Row(language='en', sentiment='NEUTRAL', count=2333867),
 Row(language='de', sentiment='NEGATIVE', count=4804),
 Row(language='de', sentiment='NEUTRAL', count=2102071),
 Row(language='de', sentiment='POSITIVE', count=98499)]

### Combining other data

In [13]:
path2 = 'file:///home/kirtan/Academics/EPFL/sem1/ADA/ADA-Project/twitter_data/gender/'
files2 = os.listdir('twitter_data/gender/')

df2 = sqlContext.read.parquet(path2 + files2[0])
df_twitter2 = df2

for f in tqdm(files2[1:]):
    temp = sqlContext.read.parquet(path2 + f)
    df_twitter2 = df_twitter2.unionAll(temp)
    
df_twitter2 = df_twitter2.dropDuplicates()

In [15]:
df_twitter2.count()

10557432

In [18]:
combined_df = df_twitter1.join(df_twitter2, df_twitter1.id == df_twitter2.id, 'inner').drop(df_twitter2.id)

In [17]:
combined_df.count()

8719750

In [44]:
combined_df = combined_df.dropDuplicates().drop(combined_df.canton)
combined_df.count()

8500030

### Adding more data

In [20]:
path3 = 'file:///home/kirtan/Academics/EPFL/sem1/ADA/ADA-Project/twitter_data/canton/'
files3 = os.listdir('twitter_data/canton')

df3 = sqlContext.read.parquet(path3 + files3[0])
df_twitter3 = df3

for f in tqdm(files3[1:]):
    temp = sqlContext.read.parquet(path3 + f)
    df_twitter3 = df_twitter3.unionAll(temp)

df_twitter3 = df_twitter3.dropDuplicates()
final = main_df.join(df_twitter3, combined_df.id == df_twitter3.id, 'left').drop(df_twitter3.id)

In [35]:
final.count()

8500030

In [26]:
final.groupBy(['language', 'sentiment']).count().sort(col("language").desc()).collect()

[Row(language='fr', sentiment='NEUTRAL', count=1963897),
 Row(language='fr', sentiment='POSITIVE', count=51940),
 Row(language='fr', sentiment='NEGATIVE', count=8040),
 Row(language='en', sentiment='POSITIVE', count=1387348),
 Row(language='en', sentiment='NEGATIVE', count=657896),
 Row(language='en', sentiment='NEUTRAL', count=2279445),
 Row(language='de', sentiment='NEGATIVE', count=4679),
 Row(language='de', sentiment='NEUTRAL', count=2050716),
 Row(language='de', sentiment='POSITIVE', count=96069)]

In [26]:
final = final.where((col('language') == 'en') | (col('sentiment').isin({'POSITIVE', 'NEGATIVE'})))

In [37]:
final.groupBy(['language', 'sentiment']).count().sort(col("language").desc()).collect()

[Row(language='fr', sentiment='POSITIVE', count=51940),
 Row(language='fr', sentiment='NEGATIVE', count=8040),
 Row(language='en', sentiment='POSITIVE', count=1387348),
 Row(language='en', sentiment='NEGATIVE', count=657896),
 Row(language='en', sentiment='NEUTRAL', count=2279445),
 Row(language='de', sentiment='NEGATIVE', count=4679),
 Row(language='de', sentiment='POSITIVE', count=96069)]

In [25]:
final.count()

4485417

In [26]:
print("Proportion with a canton:", final.filter(final.canton != 'null').count()/4485417)

Proportion with a canton: 0.37345312598583363


### Make dictionary
We make a dictionary out of the rows for which we have both canton name and source location to map source location to canton for the remaining rows for which we do not have canton but have source location. The dictionary is stored in small_dict.p. In the next part we simply load this dataframe and continue our analysis.

## Load saved data

In [38]:
#pickle.dump( dict_map, open( "small_dict.p", "wb" ) )
#sqlContext.parquet.write('file:///home/kirtan/Academics/EPFL/sem1/ADA/ADA-Project/twitter_data/final_twitter.parquet')

In [ ]:
final = sqlContext.read.parquet('file:///home/kirtan/Academics/EPFL/sem1/ADA/ADA-Project/twitter_data/final_twitter.parquet')
dict_map =  pickle.load( open( "small_dict.p", "rb" ) )

### Process the dataframe to get it in the form we need for visualisation
We start with getting the canton value which is not given to us for all the tweets

In [4]:
def map_location(value):
    if value in dict_map.keys():
        return(dict_map[value])
    else:
        return('null')
udfmap = udf(map_location, StringType())
final_loc = final.withColumn('canton_', udfmap("location"))
final = final_loc.drop(final_loc.canton)

In [22]:
print("Proportion with a canton:", final_loc.filter(final_loc.canton_ != 'null').count()/final_loc.count())

Proportion with a canton: 0.9803710558015007


In [25]:
len(set(dict_map.values()))

33

We have some fake cantons. We manually identify them and delete them.

In [7]:
bad = ['Baden-Württemberg', 'Bavaria', 'Kocaeli', 'Lower Silesian Voivodeship', 'Maha Sarakham', 'Rio de Janeiro', 'São Paulo', 'null']
final = final.drop(final.location)

In [29]:
final.show()

+-------------------+--------+---------+--------------------+--------------------+-------+----------+
|                 id|language|sentiment|                main|                date| gender|   canton_|
+-------------------+--------+---------+--------------------+--------------------+-------+----------+
|1451606716000000246|      en|  NEUTRAL|Gregory Larsen - ...|2016-01-01T00:04:33Z|UNKNOWN|    Zurich|
|1451606971000001136|      en|  NEUTRAL|@killMOOSEkill @W...|2016-01-01T00:04:02Z|UNKNOWN|      Vaud|
|1451611676000005396|      en|  NEUTRAL|Leave charging 20...|2016-01-01T01:27:19Z|UNKNOWN|      Vaud|
|1451612158000004904|      en|  NEUTRAL|Happy new year fr...|2016-01-01T01:35:30Z|UNKNOWN|    Glarus|
|1451612584000003207|      en|  NEUTRAL|@Yanruliu1 Accord...|2016-01-01T01:42:39Z|UNKNOWN|    Zurich|
|1451625427000008147|      en|  NEUTRAL|AccentForex | “Ca...|2016-01-01T05:16:46Z|UNKNOWN|      Vaud|
|1451637836000007370|      en|  NEUTRAL|@malcolm_fox2 Che...|2016-01-01T08:43:22Z|

In [30]:
final.groupBy(['language', 'sentiment']).count().sort(col("language").desc()).collect()

[Row(language='fr', sentiment='POSITIVE', count=51940),
 Row(language='fr', sentiment='NEGATIVE', count=8040),
 Row(language='en', sentiment='POSITIVE', count=1387348),
 Row(language='en', sentiment='NEGATIVE', count=657896),
 Row(language='en', sentiment='NEUTRAL', count=2279445),
 Row(language='de', sentiment='NEGATIVE', count=4679),
 Row(language='de', sentiment='POSITIVE', count=96069)]

In [10]:
def time_of_day(iso_time):
    time_struct = time.strptime(iso_time, "%Y-%m-%dT%H:%M:%SZ")
    hour = time_struct.tm_hour
    if 6 <= hour < 18:
        return "Day"
    else:
        return "Night"

def month(iso_time):
    time_struct = time.strptime(iso_time, "%Y-%m-%dT%H:%M:%SZ")
    return time_struct.tm_mon

def sentiment_map(sentiment):
    if(sentiment == 'NEGATIVE'):
        return(-1)
    elif(sentiment == 'POSITIVE'):
        return(1)
    elif(sentiment == 'NEUTRAL'):
        return(0)
    else:
        return(2)

In [13]:
udfTOD = udf(time_of_day, StringType())
udfMonth = udf(month, StringType())
udfSenti = udf(sentiment_map, IntegerType())

In [14]:
final = final.withColumn('time_period', udfTOD("date"))
final = final.withColumn('month', udfMonth("date"))
final = final.withColumn('sentiment_', udfSenti("sentiment"))

In [41]:
final.show()

+-------------------+--------+---------+--------------------+--------------------+-------+----------+-----------+-----+----------+
|                 id|language|sentiment|                main|                date| gender|   canton_|time_period|month|sentiment_|
+-------------------+--------+---------+--------------------+--------------------+-------+----------+-----------+-----+----------+
|1451606716000000246|      en|  NEUTRAL|Gregory Larsen - ...|2016-01-01T00:04:33Z|UNKNOWN|    Zurich|      Night|    1|         0|
|1451606971000001136|      en|  NEUTRAL|@killMOOSEkill @W...|2016-01-01T00:04:02Z|UNKNOWN|      Vaud|      Night|    1|         0|
|1451611676000005396|      en|  NEUTRAL|Leave charging 20...|2016-01-01T01:27:19Z|UNKNOWN|      Vaud|      Night|    1|         0|
|1451612158000004904|      en|  NEUTRAL|Happy new year fr...|2016-01-01T01:35:30Z|UNKNOWN|    Glarus|      Night|    1|         0|
|1451612584000003207|      en|  NEUTRAL|@Yanruliu1 Accord...|2016-01-01T01:42:39Z|U

In [17]:
final = final.drop(final.sentiment).drop(final.date)
for b in bad:
    final = final.filter(final.canton_ != b)

In [24]:
final.show()

+-------------------+--------+--------------------+-------+----------+-----------+-----+----------+---+
|                 id|language|                main| gender|   canton_|time_period|month|sentiment_|tmp|
+-------------------+--------+--------------------+-------+----------+-----------+-----+----------+---+
|1451606716000000246|      en|Gregory Larsen - ...|UNKNOWN|    Zurich|      Night|    1|         0|  1|
|1451606971000001136|      en|@killMOOSEkill @W...|UNKNOWN|      Vaud|      Night|    1|         0|  1|
|1451611676000005396|      en|Leave charging 20...|UNKNOWN|      Vaud|      Night|    1|         0|  1|
|1451612158000004904|      en|Happy new year fr...|UNKNOWN|    Glarus|      Night|    1|         0|  1|
|1451612584000003207|      en|@Yanruliu1 Accord...|UNKNOWN|    Zurich|      Night|    1|         0|  1|
|1451625427000008147|      en|AccentForex | “Ca...|UNKNOWN|      Vaud|      Night|    1|         0|  1|
|1451637836000007370|      en|@malcolm_fox2 Che...|UNKNOWN|     

In [19]:
final = final.withColumn('tmp', lit(1))
final = final.drop(final.id).drop(final.main)

In [30]:
final.show()

+--------+-------+----------+-----------+-----+----------+---+
|language| gender|   canton_|time_period|month|sentiment_|tmp|
+--------+-------+----------+-----------+-----+----------+---+
|      en|UNKNOWN|    Zurich|      Night|    1|         0|  1|
|      en|UNKNOWN|      Vaud|      Night|    1|         0|  1|
|      en|UNKNOWN|      Vaud|      Night|    1|         0|  1|
|      en|UNKNOWN|    Glarus|      Night|    1|         0|  1|
|      en|UNKNOWN|    Zurich|      Night|    1|         0|  1|
|      en|UNKNOWN|      Vaud|      Night|    1|         0|  1|
|      en|UNKNOWN|      Vaud|        Day|    1|         0|  1|
|      en|UNKNOWN|      Vaud|        Day|    1|         0|  1|
|      en|UNKNOWN|    Zurich|        Day|    1|         0|  1|
|      en|   MALE|    Geneva|        Day|    1|         0|  1|
|      en|UNKNOWN|Basel-City|        Day|    1|         0|  1|
|      en|UNKNOWN|      Vaud|        Day|    1|         0|  1|
|      en| FEMALE|      Vaud|        Day|    1|        

In [28]:
#final.write.parquet('file:///home/kirtan/Academics/EPFL/sem1/ADA/ADA-Project/twitter_data/final_twitter1.parquet')

### Load prepared dataframe

In [4]:
final = sqlContext.read.parquet('file:///home/kirtan/Academics/EPFL/sem1/ADA/ADA-Project/twitter_data/final_twitter2.parquet')

In [5]:
final.show()

+--------+-------+----------+-----------+-----+----------+---+
|language| gender|   canton_|time_period|month|sentiment_|tmp|
+--------+-------+----------+-----------+-----+----------+---+
|      en|UNKNOWN|      Vaud|      Night|    1|         0|  1|
|      en|UNKNOWN|      Vaud|      Night|    1|         0|  1|
|      en|UNKNOWN|    Ticino|      Night|    1|         0|  1|
|      en| FEMALE|      Vaud|      Night|    1|         0|  1|
|      en|   MALE|      Vaud|      Night|    1|         0|  1|
|      en|UNKNOWN|      Vaud|      Night|    1|         0|  1|
|      en|UNKNOWN|      Vaud|        Day|    1|         0|  1|
|      en|   MALE|      Vaud|        Day|    1|         0|  1|
|      en|UNKNOWN|    Geneva|        Day|    1|         0|  1|
|      en|UNKNOWN|      Vaud|        Day|    1|         0|  1|
|      en|UNKNOWN|    Geneva|        Day|    1|         0|  1|
|      en|UNKNOWN|      Vaud|        Day|    1|         0|  1|
|      en|UNKNOWN|Basel-City|        Day|    1|        

In [7]:
final = final.groupBy('canton_', 'month', 'language', 'gender', 'time_period').agg((func.sum('sentiment_')/func.sum('tmp')).alias('avg_sentiment'), func.sum('tmp').alias('count'))

In [8]:
final.show()

+--------------------+-----+--------+-------+-----------+-------------------+-----+
|             canton_|month|language| gender|time_period|      avg_sentiment|count|
+--------------------+-----+--------+-------+-----------+-------------------+-----+
|          Basel-City|    7|      en|   MALE|        Day|0.16316931982633864| 2764|
|              Geneva|    8|      en|   MALE|        Day|0.13686382393397525|13086|
|           Solothurn|    9|      en| FEMALE|      Night| 0.2608695652173913|   69|
|              Schwyz|    5|      en|UNKNOWN|        Day|                0.5|    6|
|          Basel-City|    9|      fr|UNKNOWN|      Night|                1.0|    2|
|             Thurgau|    8|      en|   MALE|        Day|0.10526315789473684|   19|
|    Basel-Landschaft|    5|      en| FEMALE|        Day|                0.2|    5|
|              Glarus|    4|      en|   MALE|        Day|                1.0|    1|
|          Basel-City|    1|      de|UNKNOWN|        Day| 0.8735042735042735

In [9]:
# Rename Columns
final = final.selectExpr("canton_ as canton", "month as month", "language as language", "gender as gender", "time_period as time_period",
                  "avg_sentiment as avg_sentiment", "count as count")

In [10]:
final.show()

+--------------------+-----+--------+-------+-----------+-------------------+-----+
|              canton|month|language| gender|time_period|      avg_sentiment|count|
+--------------------+-----+--------+-------+-----------+-------------------+-----+
|          Basel-City|    7|      en|   MALE|        Day|0.16316931982633864| 2764|
|              Geneva|    8|      en|   MALE|        Day|0.13686382393397525|13086|
|           Solothurn|    9|      en| FEMALE|      Night| 0.2608695652173913|   69|
|              Schwyz|    5|      en|UNKNOWN|        Day|                0.5|    6|
|          Basel-City|    9|      fr|UNKNOWN|      Night|                1.0|    2|
|             Thurgau|    8|      en|   MALE|        Day|0.10526315789473684|   19|
|    Basel-Landschaft|    5|      en| FEMALE|        Day|                0.2|    5|
|              Glarus|    4|      en|   MALE|        Day|                1.0|    1|
|          Basel-City|    1|      de|UNKNOWN|        Day| 0.8735042735042735

In [11]:
#final.write.parquet('file:///home/kirtan/Academics/EPFL/sem1/ADA/ADA-Project/twitter_data/final_twitter3.parquet')

In [12]:
final.count()

2448

In [13]:
fp = final.toPandas()

In [14]:
# Dataframe to be used for visualisation
fp.to_csv('twitter_data/twitter.csv', sep=',', encoding='utf-8')